This is a notebook that converts data in LeetCode SQL 50 to spark dataframe and solves the same question using spark

In [1]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/20 22:02:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def convert_pandas_to_spark_df(df: pd.DataFrame) -> None:
    return spark.createDataFrame(df)

## Q1) 1757. Recyclable and Low Fat Products

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in any order.

The result format is in the following example.

### Products Table

| product_id | low_fats | recyclable |
|------------|----------|------------|
| 0          | Y        | N          |
| 1          | Y        | Y          |
| 2          | N        | Y          |
| 3          | Y        | Y          |
| 4          | N        | N          |

### Output Table

| product_id |
|------------|
| 1          |
| 3          |

In [18]:
data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]
products = pd.DataFrame(data, columns=['product_id', 'low_fats', 'recyclable']).astype({'product_id':'int64', 'low_fats':'category', 'recyclable':'category'})

In [19]:
q1_df = convert_pandas_to_spark_df(products)

In [20]:
q1_df.show()

+----------+--------+----------+
|product_id|low_fats|recyclable|
+----------+--------+----------+
|         0|       Y|         N|
|         1|       Y|         Y|
|         2|       N|         Y|
|         3|       Y|         Y|
|         4|       N|         N|
+----------+--------+----------+



In [21]:
res = q1_df \
    .filter((q1_df['low_fats'] == 'Y') & (q1_df['recyclable'] == 'Y')) \
    .select('product_id')

res.show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+

